# Next Word Prediction

**importing necessary libraries**

In [1]:
import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import heapq

**data is in text format, so we can simply read it**

In [60]:
path = '/content/1661-0.txt'
text = open(path, encoding="utf8").read().lower()
print('total words:', len(text))

total words: 581888


**we are tokenizing each sentence for a single word**

In [3]:
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)

In [4]:
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

**we are gonna predict single next word from previous 5 words**

In [61]:
WORD_LENGTH = 5
prev_words = []
next_words = []
for i in range(len(words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])
print(prev_words[0])
print(next_words[0])

['project', 'gutenberg', 's', 'the', 'adventures']
of


**creating features and labels from data**

In [6]:
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1

### Network
**we are using recurrent neural network for better understanding**

In [48]:
model = Sequential()
model.add(LSTM(64,  input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

In [62]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='/content/final_model.h5', 
                               verbose=1, save_best_only=True)


optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=2, callbacks=[checkpointer], shuffle=True).history

Epoch 1/2
811/811 [==============================] - 11s 12ms/step - loss: 5.7320 - accuracy: 0.1422 - val_loss: 8.0924 - val_accuracy: 0.1045

Epoch 00001: val_loss improved from inf to 8.09236, saving model to /content/final_model.h5
Epoch 2/2
811/811 [==============================] - 9s 11ms/step - loss: 5.8472 - accuracy: 0.1638 - val_loss: 8.1155 - val_accuracy: 0.1045

Epoch 00002: val_loss did not improve from 8.09236


In [63]:
model = load_model('/content/final_model.h5')

### preprocessing the input sentence

In [64]:
def prepare_input(text):
    x = np.zeros((1, WORD_LENGTH, len(unique_words)))
    for t, word in enumerate(text.split()):
        print(word)
        x[0, t, unique_word_index[word]] = 1
    return x
prepare_input("whenever we go to".lower())

whenever
we
go
to


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [65]:
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    return heapq.nlargest(top_n, range(len(preds)), preds.take)

In [66]:
def predict_completions(text, n=3):
    if text == "":
        return("0")
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [unique_words[idx] for idx in next_indices]

### final output

In [86]:
q =  "when i woke up they"
print("correct sentence: ",q)
seq = " ".join(tokenizer.tokenize(q.lower())[0:5])
print("Sequence: ",seq)
print("next possible words: ", predict_completions(seq, 5))

correct sentence:  when i woke up they
Sequence:  when i woke up they
when
i
woke
up
they
next possible words:  ['were', 'are', 'have', 'had', 'can']
